In [ ]:
# Following https://github.com/huggingface/notebooks/blob/master/examples/translation.ipynb

In [ ]:
# Load the TensorBoard notebook extension
%load_ext tensorboard
# Clear any logs from previous runs
!rm -rf ./logs/ 

In [ ]:
from google.colab import drive
drive.mount('drive')

Mounted at drive


In [ ]:
!unzip lc-quad-wikidata-2021-07-09.zip

Archive:  lc-quad-wikidata-2021-07-09.zip
 extracting: lc-quad-wikidata-2021-07-09/dataset_dict.json  
   creating: lc-quad-wikidata-2021-07-09/test/
  inflating: lc-quad-wikidata-2021-07-09/test/dataset.arrow  
  inflating: lc-quad-wikidata-2021-07-09/test/dataset_info.json  
  inflating: lc-quad-wikidata-2021-07-09/test/state.json  
   creating: lc-quad-wikidata-2021-07-09/train/
  inflating: lc-quad-wikidata-2021-07-09/train/dataset.arrow  
  inflating: lc-quad-wikidata-2021-07-09/train/dataset_info.json  
  inflating: lc-quad-wikidata-2021-07-09/train/state.json  


In [ ]:
!nvidia-smi

Mon Aug  9 09:22:14 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.42.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   49C    P8    30W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
model_checkpoint = 't5-small'
gdir = 'drive/My Drive/Colab Notebooks/t5-2021-07-11/'
model_name='sparql-translator-t5-2021-08-08-large'
model_path='./models/'+model_name
ds_path= 'lc-quad-wikidata-2021-07-09'

In [ ]:
!mkdir models

In [ ]:
!pip install datasets transformers sacrebleu -qqq

In [ ]:
from datasets import load_dataset, load_metric, Dataset, load_from_disk
raw_datasets = load_from_disk(ds_path)

In [ ]:
!pip install tqdm==4.49.0

     |████████████████████████████████| 69 kB 3.0 MB/s 
  Attempting uninstall: tqdm
    Found existing installation: tqdm 4.62.0
    Uninstalling tqdm-4.62.0:
      Successfully uninstalled tqdm-4.62.0


In [ ]:
from tqdm import tqdm

In [ ]:
metric = load_metric("sacrebleu")

Downloading:   0%|          | 0.00/2.38k [00:00<?, ?B/s]

In [ ]:
# Preprocessing

In [ ]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

Downloading:   0%|          | 0.00/1.20k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/792k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

In [ ]:
print(raw_datasets['test']['translation'][0]['sparql'])

select distinct ?sbj where [ ?sbj wdt:head_of_state wd:mahmoud_abbas . ?sbj wdt:instance_of wd:country ]


In [ ]:
tokenizer.decode(tokenizer(raw_datasets['test']['translation'][0]['sparql'])['input_ids'])

'select distinct?sbj where [?sbj wdt:head_of_state wd:mahmoud_abbas.?sbj wdt:instance_of wd:country ]</s>'

In [ ]:
print(raw_datasets['test']['translation'][0]['en'],tokenizer(raw_datasets['test']['translation'][0]['en']))

Who is the  {country} for {head of state} of {Mahmoud Abbas} {'input_ids': [2645, 19, 8, 3, 2, 17529, 2, 21, 3, 2, 3313, 13, 538, 2, 13, 3, 2, 329, 9, 107, 51, 1063, 26, 891, 4883, 2, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}


In [ ]:
prefix = "translate English to Sparql: "

In [ ]:
max_input_length = 0 
max_target_length = 0
for d in tqdm(raw_datasets['train']['translation']):
    len_en = len(d['en'])
    len_qry = len(d['sparql'])
    if len_en > max_input_length: max_input_length=len_en
    if len_qry > max_target_length: max_target_length=len_qry

100%|██████████| 38175/38175 [00:00<00:00, 1073519.33it/s]


In [ ]:
print(max_input_length, max_target_length)

248 349


In [ ]:
source_lang = "en"
target_lang = "sparql"

def preprocess_function(examples):
    inputs = []
    targets= []
    for ex in examples["translation"]:
      inputs.append(prefix + ex[source_lang])
      targets.append(ex[target_lang])
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)

    # Setup the tokenizer for targets
    with tokenizer.as_target_tokenizer():
      labels = tokenizer(targets, max_length=max_target_length, truncation=True)
      
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [ ]:
tokenized_datasets = raw_datasets.map(preprocess_function, batched=True)

  0%|          | 0/39 [00:00<?, ?ba/s]

  0%|          | 0/10 [00:00<?, ?ba/s]

In [ ]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['attention_mask', 'input_ids', 'labels', 'translation'],
        num_rows: 38175
    })
    test: Dataset({
        features: ['attention_mask', 'input_ids', 'labels', 'translation'],
        num_rows: 9485
    })
})

In [ ]:
# Fine-tuning the model

In [ ]:
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer
model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

Downloading:   0%|          | 0.00/242M [00:00<?, ?B/s]

In [ ]:
batch_size = 8
args = Seq2SeqTrainingArguments(
    model_name,
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=1,
    predict_with_generate=True,
    fp16=True,
)

In [ ]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [ ]:
fake_preds = ["hello there", "general kenobi"]
fake_labels = [["hello there"], ["general kenobi"]]
metric.compute(predictions=fake_preds, references=fake_labels)

{'bp': 1.0,
 'counts': [4, 2, 0, 0],
 'precisions': [100.0, 100.0, 0.0, 0.0],
 'ref_len': 4,
 'score': 0.0,
 'sys_len': 4,
 'totals': [4, 2, 0, 0]}

In [ ]:
# The last thing to define for our Seq2SeqTrainer is how to compute 
# the metrics from the predictions. We need to define 
# a function for this, which will just use the metric we loaded earlier, 
# and we have to do a bit of pre-processing to decode the predictions into texts:

import numpy as np

def postprocess_text(preds, labels):
    preds = [pred.replace('?',' ?').replace('.', ' .').strip() for pred in preds]
    labels = [[label.replace('?',' ?').replace('.', ' .').strip()] for label in labels]

    return preds, labels

def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Some simple post-processing
    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

    result = metric.compute(predictions=decoded_preds, references=decoded_labels)
    result = {"bleu": result["score"]}

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
    result["gen_len"] = np.mean(prediction_lens)
    result = {k: round(v, 4) for k, v in result.items()}
    return result

In [ ]:
trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

Using amp fp16 backend


In [ ]:
!tensorboard dev upload --logdir runs

2021-08-09 09:24:54.619090: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0

***** TensorBoard Uploader *****

This will upload your TensorBoard logs to https://tensorboard.dev/ from
the following directory:

runs

This TensorBoard will be visible to everyone. Do not upload sensitive
data.

Your use of this service is subject to Google's Terms of Service
<https://policies.google.com/terms> and Privacy Policy
<https://policies.google.com/privacy>, and TensorBoard.dev's Terms of Service
<https://tensorboard.dev/policy/terms/>.

This notice will not be shown again while you are logged into the uploader.
To log out, run `tensorboard dev auth revoke`.

Continue? (yes/NO) yes

Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=373649185512-8v619h5kft38l4456nm2dj4ubeqsrvh6.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scop

In [ ]:
%%time
trainer.train()

The following columns in the training set  don't have a corresponding argument in `T5ForConditionalGeneration.forward` and have been ignored: translation.
***** Running training *****
  Num examples = 38175
  Num Epochs = 1
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 4772


Epoch,Training Loss,Validation Loss,Bleu,Gen Len
1,0.711500,0.508446,3.549400,19.000000


/usr/local/lib/python3.7/dist-packages/transformers/trainer.py:1310: FutureWarning: Non-finite norm encountered in torch.nn.utils.clip_grad_norm_; continuing anyway. Note that the default behavior will change in a future release to error out if a non-finite total norm is encountered. At that point, setting error_if_nonfinite=false will be required to retain the old behavior.
  args.max_grad_norm,
Saving model checkpoint to sparql-translator-t5-2021-08-08-large/checkpoint-500
Configuration saved in sparql-translator-t5-2021-08-08-large/checkpoint-500/config.json
Model weights saved in sparql-translator-t5-2021-08-08-large/checkpoint-500/pytorch_model.bin
tokenizer config file saved in sparql-translator-t5-2021-08-08-large/checkpoint-500/tokenizer_config.json
Special tokens file saved in sparql-translator-t5-2021-08-08-large/checkpoint-500/special_tokens_map.json
Saving model checkpoint to sparql-translator-t5-2021-08-08-large/checkpoint-1000
Configuration saved in sparql-translator-t5-2

CPU times: user 36min 54s, sys: 10min 50s, total: 47min 45s
Wall time: 47min 42s


TrainOutput(global_step=4772, training_loss=1.0800021827870423, metrics={'train_runtime': 2862.8312, 'train_samples_per_second': 13.335, 'train_steps_per_second': 1.667, 'total_flos': 1518701282033664.0, 'train_loss': 1.0800021827870423, 'epoch': 1.0})

In [ ]:
trainer.save_model(model_path)

Saving model checkpoint to ./models/sparql-translator-t5-2021-08-08-large
Configuration saved in ./models/sparql-translator-t5-2021-08-08-large/config.json
Model weights saved in ./models/sparql-translator-t5-2021-08-08-large/pytorch_model.bin
tokenizer config file saved in ./models/sparql-translator-t5-2021-08-08-large/tokenizer_config.json
Special tokens file saved in ./models/sparql-translator-t5-2021-08-08-large/special_tokens_map.json


In [ ]:
!ls -l --block-size=M {model_path}

total 233M
-rw-r--r-- 1 root root   1M Aug  9 10:16 config.json
-rw-r--r-- 1 root root 231M Aug  9 10:16 pytorch_model.bin
-rw-r--r-- 1 root root   1M Aug  9 10:16 special_tokens_map.json
-rw-r--r-- 1 root root   1M Aug  9 10:16 tokenizer_config.json
-rw-r--r-- 1 root root   2M Aug  9 10:16 tokenizer.json
-rw-r--r-- 1 root root   1M Aug  9 10:16 training_args.bin


In [ ]:
!mkdir drive/MyDrive/models/{model_path}

mkdir: cannot create directory ‘drive/MyDrive/models/./models/sparql-translator-t5-2021-08-08-large’: No such file or directory


In [ ]:
!cp {model_path}/* drive/MyDrive/models/{model_path}/

cp: target 'drive/MyDrive/models/./models/sparql-translator-t5-2021-08-08-large/' is not a directory
